In [88]:
import optimizers as opt
import helper_functions as helper

In [ ]:
"""
Circle
init_pheromone = 0.2
test.main_loop(num_iter=50, alpha=1, beta=4, rho=1e-1, Q=6)

TSP1
test.main_loop(num_iter=50, alpha=1, beta=5, rho=1e-1, Q=564)
"""
graph, perim = helper.create_circle(400)
test = opt.ACO(graph, init_pheromone=0.2)
dist, route = test.main_loop_plot(num_iter=25, alpha=1, beta=4, rho=2e-1, Q=6, gif_name="circle_400_ACO")
print("Min dist: %.3f\n Real min dist: %.3f" % (dist, perim))

In [ ]:
"""
tsp131 - 564
tsp237 - 1019
"""

# graph, perim = create_circle(100)
file_path = r"datasets\tsp131.tsp"
graph = helper.read_tsp_file(file_path)
test = opt.GO(130, graph)
dist, route = test.main_loop_plot(1000, 130, 0.4, gif_name="test")
# print("Min dist: %.3f\n Real min dist: %.3f" % (dist, perim))